# v1 전처리

In [1]:
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/HD dacon2")
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/Colab Notebooks/HD dacon2'

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import lightgbm as lgb
import bisect
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

%matplotlib inline
plt.style.use(['dark_background'])
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [34]:
# 데이터 불러오기
train = pd.read_csv('train.csv').drop(columns=['SAMPLE_ID'])
test = pd.read_csv('test.csv').drop(columns=['SAMPLE_ID'])

In [35]:
#  1. CI_HOUR 컬럼에서 표준편차 + 2 * 표준편차 이상이 되는 부분 이상치라고 판단 후, 삭제

# 'CI_HOUR' 열의 평균과 표준 편차 계산
mean = train['CI_HOUR'].mean()
std = train['CI_HOUR'].std()

# 이상치 경계 설정 (예: 평균에서 2배 표준 편차를 벗어나는 값)
lower_bound = mean - 2 * std
upper_bound = mean + 2 * std

# 이상치를 제거하고 정상 범위의 데이터만 남김
#filtered_train = train[(train['CI_HOUR'] >= lower_bound) & (train['CI_HOUR'] <= upper_bound)]
train = train[(train['CI_HOUR'] >= lower_bound) & (train['CI_HOUR'] <= upper_bound)].reset_index(drop=True)

In [36]:
'''# DIST 및 BUILT 컬럼의 평균과 표준편차를 계산합니다.
mean_dist = train['DIST'].mean()
std_dist = train['DIST'].std()
mean_built = train['BUILT'].mean()
std_built = train['BUILT'].std()

# 이상치 경계 설정 (예: 평균에서 2배 표준 편차를 벗어나는 값)
lower_bound_dist = mean_dist - 3 * std_dist
upper_bound_dist = mean_dist + 3 * std_dist


lower_bound_built = mean_built - 3 * std_built
upper_bound_built = mean_built + 3 * std_built

# 조건을 만족하는 값만 대체합니다.
train = train[(train['DIST'] >= lower_bound_dist) & (train['DIST'] <= upper_bound_dist)].reset_index(drop=True)
train = train[(train['BUILT'] >= lower_bound_built) & (train['BUILT'] <= upper_bound_built)].reset_index(drop=True)'''

"# DIST 및 BUILT 컬럼의 평균과 표준편차를 계산합니다.\nmean_dist = train['DIST'].mean()\nstd_dist = train['DIST'].std()\nmean_built = train['BUILT'].mean()\nstd_built = train['BUILT'].std()\n\n# 이상치 경계 설정 (예: 평균에서 2배 표준 편차를 벗어나는 값)\nlower_bound_dist = mean_dist - 3 * std_dist\nupper_bound_dist = mean_dist + 3 * std_dist\n\n\nlower_bound_built = mean_built - 3 * std_built\nupper_bound_built = mean_built + 3 * std_built\n\n# 조건을 만족하는 값만 대체합니다.\ntrain = train[(train['DIST'] >= lower_bound_dist) & (train['DIST'] <= upper_bound_dist)].reset_index(drop=True)\ntrain = train[(train['BUILT'] >= lower_bound_built) & (train['BUILT'] <= upper_bound_built)].reset_index(drop=True)"

In [37]:
# 2. 다중공선성 문제 해결을 위해 Length 컬럼 이외 삭제 (BREADTH, DEADWEIGHT, DEPTH, DRAUGHT, GT)

for df in [train, test]:
    df.drop(columns = 'BREADTH', inplace = True)
    df.drop(columns = 'DEADWEIGHT', inplace = True)
    # df.drop(columns = 'DEPTH', inplace = True)
    df.drop(columns = 'DRAUGHT', inplace = True)
    df.drop(columns = 'GT', inplace = True)

In [38]:
# 데이터프레임 전체 결측치 파악
total_missing = train.isna().sum()
print(total_missing)

ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ATA                        0
ID                         0
BUILT                      0
DEPTH                      1
LENGTH                     1
SHIPMANAGER                0
FLAG                       0
U_WIND                159187
V_WIND                159187
AIR_TEMPERATURE       160102
BN                    159187
ATA_LT                     0
PORT_SIZE                  0
CI_HOUR                    0
dtype: int64


In [39]:
'''
from sklearn.impute import KNNImputer

### 결측치 KNNImputer를 통해 대체

# K-NN 대체기
knn_imputer = KNNImputer(n_neighbors=10)

# K-NN 대체를 적용할 열을 지정
columns_to_impute = ['U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BN']

# train 데이터프레임에서 결측치를 K-NN으로 대체
filtered_train[columns_to_impute] = knn_imputer.fit_transform(filtered_train[columns_to_impute])

filtered_train = pd.DataFrame(filtered_train, columns=filtered_train.columns)
#filtered_train.to_csv('train_imputed.csv', index = False)

'''
# 결측치 많은 컬럼 제거
train.drop(columns = 'U_WIND', inplace = True)
train.drop(columns = 'V_WIND', inplace = True)
train.drop(columns = 'AIR_TEMPERATURE', inplace = True)
train.drop(columns = 'BN', inplace = True)

test.drop(columns = 'U_WIND', inplace = True)
test.drop(columns = 'V_WIND', inplace = True)
test.drop(columns = 'AIR_TEMPERATURE', inplace = True)
test.drop(columns = 'BN', inplace = True)


In [40]:
# 4. 결측치가 존재하는 데이터 1개 삭제
train = train.dropna(subset=['LENGTH']).reset_index(drop=True)

In [41]:
# 5. DIST가 0이면, CI_HOUR 모두 0으로 변경
train.loc[train['DIST'] == 0, 'CI_HOUR'] = 0

print(train)

       ARI_CO ARI_PO SHIP_TYPE_CATEGORY       DIST               ATA       ID  \
0          SG   GIW5          Container  30.881018  2018-12-17 21:29  Z618338   
1          IN   UJM2               Bulk   0.000000   2014-09-23 6:59  X886125   
2          CN   EUC8          Container   0.000000  2015-02-03 22:00  T674582   
3          JP   ZAG4          Container   0.000000   2020-01-17 4:02  Y847238   
4          SG   GIW5          Container  27.037650   2020-01-26 7:51  A872328   
...       ...    ...                ...        ...               ...      ...   
379811     JP   QYY1          Container   0.000000   2017-06-06 5:02  Y375615   
379812     SG   GIW5               Bulk   5.884603   2019-10-16 0:36  K635567   
379813     US   QGN3               Bulk  70.660241  2021-03-23 22:35  J284147   
379814     TW   JWI3          Container   9.448179   2015-01-08 7:15  J644215   
379815     TW   JWI3          Container   0.000000  2015-06-08 23:30  D123358   

        BUILT  DEPTH  LENGT

In [42]:
# 6. year, month, day, hour, minute, weekday 컬럼 그대로 생성
# datetime 컬럼 처리
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])


# datetime을 여러 파생 변수로 변환
for df in [train, test]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday

'''
train['CI_HOUR'] = pd.to_timedelta(train['CI_HOUR'], unit="hours")
train['Berthing datetime'] = train['ATA'] + train['CI_HOUR']
test['CI_HOUR'] = pd.to_timedelta(test['CI_HOUR'], unit="hours")
test['Berthing datetime'] = test['ATA'] + test['CI_HOUR']
'''

# 7. datetime 컬럼 제거
train.drop(columns='ATA', inplace=True)
test.drop(columns='ATA', inplace=True)

'''# 결측치 많은 컬럼 제거
train.drop(columns = 'U_WIND', inplace = True)
train.drop(columns = 'V_WIND', inplace = True)
train.drop(columns = 'AIR_TEMPERATURE', inplace = True)
train.drop(columns = 'BN', inplace = True)

test.drop(columns = 'U_WIND', inplace = True)
test.drop(columns = 'V_WIND', inplace = True)
test.drop(columns = 'AIR_TEMPERATURE', inplace = True)
test.drop(columns = 'BN', inplace = True)'''

# 8. Categorical 컬럼 인코딩
categorical_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
    le = LabelEncoder()
    train[feature] = le.fit_transform(train[feature].astype(str))
    le_classes_set = set(le.classes_)
    test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
    le_classes = le.classes_.tolist()
    bisect.insort_left(le_classes, '-1')
    le.classes_ = np.array(le_classes)
    test[feature] = le.transform(test[feature].astype(str))
    encoders[feature] = le


'''# 결측치 처리
train.fillna(train.mean(), inplace=True)
test.fillna(train.mean(), inplace=True)
'''

Encoding features: 100%|██████████| 6/6 [00:01<00:00,  4.01it/s]


'# 결측치 처리\ntrain.fillna(train.mean(), inplace=True)\ntest.fillna(train.mean(), inplace=True)\n'

In [43]:
# Frequency Encoding
frequency_SHIP_TYPE_CATEGORY = train.groupby("SHIP_TYPE_CATEGORY").size()/len(train)
train['frequency_SHIP_TYPE_CATEGORY'] = train['SHIP_TYPE_CATEGORY'].map(frequency_SHIP_TYPE_CATEGORY)


In [44]:
# Frequency Encoding
frequency_SHIP_TYPE_CATEGORY = test.groupby("SHIP_TYPE_CATEGORY").size()/len(test)
test['frequency_SHIP_TYPE_CATEGORY'] = test['SHIP_TYPE_CATEGORY'].map(frequency_SHIP_TYPE_CATEGORY)


In [45]:
frequency_SHIP_TYPE_CATEGORY

SHIP_TYPE_CATEGORY
1    0.419432
2    0.103927
3    0.424022
4    0.052619
dtype: float64

In [46]:
train

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BUILT,DEPTH,LENGTH,SHIPMANAGER,FLAG,ATA_LT,PORT_SIZE,CI_HOUR,year,month,day,hour,minute,weekday,frequency_SHIP_TYPE_CATEGORY
0,17,21,2,30.881018,24456,24,10.0,180.0,679,86,5,0.002615,3.450000,2018,12,17,21,29,0,0.427597
1,7,81,0,0.000000,22899,13,10.0,180.0,4669,71,12,0.000217,0.000000,2014,9,23,6,59,1,0.418068
2,4,14,2,0.000000,18803,12,30.0,370.0,1356,70,6,0.001614,0.000000,2015,2,3,22,0,1,0.427597
3,8,101,2,0.000000,23799,18,10.0,120.0,3769,4,13,0.000356,0.000000,2020,1,17,4,2,4,0.427597
4,17,21,2,27.037650,903,10,20.0,300.0,1667,64,15,0.002615,253.554444,2020,1,26,7,51,6,0.427597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379811,8,66,2,0.000000,23245,27,10.0,110.0,2530,18,14,0.000552,0.000000,2017,6,6,5,2,1,0.427597
379812,17,21,0,5.884603,10082,12,10.0,80.0,2371,121,8,0.002615,144.061389,2019,10,16,0,36,2,0.418068
379813,21,61,0,70.660241,8724,8,20.0,200.0,6038,99,18,0.000155,41.482222,2021,3,23,22,35,1,0.418068
379814,19,35,2,9.448179,9146,29,10.0,170.0,1749,20,15,0.000990,7.485278,2015,1,8,7,15,3,0.427597


# DNN

In [47]:
train

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BUILT,DEPTH,LENGTH,SHIPMANAGER,FLAG,ATA_LT,PORT_SIZE,CI_HOUR,year,month,day,hour,minute,weekday,frequency_SHIP_TYPE_CATEGORY
0,17,21,2,30.881018,24456,24,10.0,180.0,679,86,5,0.002615,3.450000,2018,12,17,21,29,0,0.427597
1,7,81,0,0.000000,22899,13,10.0,180.0,4669,71,12,0.000217,0.000000,2014,9,23,6,59,1,0.418068
2,4,14,2,0.000000,18803,12,30.0,370.0,1356,70,6,0.001614,0.000000,2015,2,3,22,0,1,0.427597
3,8,101,2,0.000000,23799,18,10.0,120.0,3769,4,13,0.000356,0.000000,2020,1,17,4,2,4,0.427597
4,17,21,2,27.037650,903,10,20.0,300.0,1667,64,15,0.002615,253.554444,2020,1,26,7,51,6,0.427597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379811,8,66,2,0.000000,23245,27,10.0,110.0,2530,18,14,0.000552,0.000000,2017,6,6,5,2,1,0.427597
379812,17,21,0,5.884603,10082,12,10.0,80.0,2371,121,8,0.002615,144.061389,2019,10,16,0,36,2,0.418068
379813,21,61,0,70.660241,8724,8,20.0,200.0,6038,99,18,0.000155,41.482222,2021,3,23,22,35,1,0.418068
379814,19,35,2,9.448179,9146,29,10.0,170.0,1749,20,15,0.000990,7.485278,2015,1,8,7,15,3,0.427597


In [48]:
X_train = train.drop(columns = 'CI_HOUR')
y_train = train['CI_HOUR']

X_test = test[X_train.columns]

In [49]:
# 정규화 진행
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Min-Max 스케일러 초기화 및 스케일링 적용
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [50]:
len(X_train_scaled[1])

19

In [51]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 모델
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(19,)),  # 입력 레이어
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # 출력 레이어 (회귀 모델이므로 하나의 출력 뉴런)
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_absolute_error')

# 모델 학습
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])
# model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# 모델 평가
loss = model.evaluate(X_train_scaled, y_train)
print("평가 손실:", loss)



Epoch 1/50
9496/9496 [==============================] - 17s 2ms/step - loss: 25.9338 - val_loss: 24.9946
Epoch 2/50
9496/9496 [==============================] - 16s 2ms/step - loss: 24.9048 - val_loss: 24.8029
Epoch 3/50
9496/9496 [==============================] - 16s 2ms/step - loss: 24.6633 - val_loss: 24.5935
Epoch 4/50
9496/9496 [==============================] - 16s 2ms/step - loss: 24.5187 - val_loss: 24.5189
Epoch 5/50
9496/9496 [==============================] - 16s 2ms/step - loss: 24.3934 - val_loss: 24.4939
Epoch 6/50
9496/9496 [==============================] - 16s 2ms/step - loss: 24.2986 - val_loss: 24.4733
Epoch 7/50
9496/9496 [==============================] - 16s 2ms/step - loss: 24.1973 - val_loss: 24.2162
Epoch 8/50
9496/9496 [==============================] - 16s 2ms/step - loss: 24.1083 - val_loss: 24.2149
Epoch 9/50
9496/9496 [==============================] - 17s 2ms/step - loss: 24.0381 - val_loss: 24.2412
Epoch 10/50
9496/9496 [==============================] 

In [52]:
 # 예측
y_pred = model.predict(X_test_scaled)

6891/6891 [==============================] - 7s 1ms/step


In [53]:
submit = pd.read_csv('./sample_submission.csv')
submit['CI_HOUR'] = y_pred
submit.to_csv('./DNN19_submit.csv', index=False)